In [17]:
# AI Model Implementation in Jupyter Notebook

## Cell 1: Import necessary libraries
import pandas as pd
import numpy as np
import joblib  # For saving and loading the model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [ ]:
## Cell 2: Load the updated dataset
# Load the dataset from Excel file
dataset_path = r"C:\Users\HP\Documents\GitHub\vroomble\Vroomble Dataset\vroomble_car_datasets.xlsx"
sheets = pd.read_excel(dataset_path, sheet_name=None)

# Extract relevant sheets
car_model_data = sheets["Car_Model"]
car_modifications_data = sheets["Car_Modifications"]

# Display first few rows to check if the dataset is loaded correctly
print("✅ Car Model Data Preview:")
print(car_model_data.head())

print("\n✅ Car Modifications Data Preview:")
print(car_modifications_data.head())



FileNotFoundError: [Errno 2] No such file or directory: '../Vroomble Dataset/vroomble_car_datasets.xlsx'

In [ ]:
## Cell 3: Prepare training data for the AI model
# Initialize empty lists for training data
X = []  # Features (Base Price, Modification Costs, Inflation Rate)
y = []  # Target (Predicted Price)

# Iterate through car models
for index, row in car_model_data.iterrows():
    base_price = row["Base_Price_PHP"]
    inflation_rate = row["Monthly_Inflation_Rate"]
    
    # Get modification costs for this model
    modifications = car_modifications_data[car_modifications_data["Model"] == row["Model"]]
    total_modification_cost = modifications["Modification_Cost_PHP"].sum()
    
    # Append feature set and target value
    X.append([base_price, total_modification_cost, inflation_rate])
    y.append(base_price + total_modification_cost)

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

print("✅ Training data prepared successfully!")
print("\n🔹 Sample Features (X):", X[:5])  # Show first 5 feature sets
print("\n🔹 Sample Target (y):", y[:5])  # Show first 5 target values




In [ ]:
## Cell 4: Convert to NumPy arrays and train the model
# Normalize features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train the model
model = LinearRegression()
model.fit(X_scaled, y)

print("✅ Model training completed!")
print("\n🔹 Intercept:", model.intercept_)
print("\n🔹 Coefficients:", model.coef_)



In [ ]:
## Cell 5: Save the trained model and scaler for FastAPI backend usage
# Save the trained model and scaler for FastAPI backend usage
joblib.dump(model, "trained_model.pkl")
joblib.dump(scaler, "scaler.pkl")

print("✅ Model and scaler saved successfully!")


In [ ]:
## Cell 6: Function to predict car price based on selected parts
# Function to predict car price based on selected parts
def predict_price(model_name, selected_parts, months=12):
    available_models = car_model_data["Model"].unique().tolist()

    if model_name not in available_models:
        print(f"❌ Error: Model '{model_name}' not found.")
        return None

    # Get car details (Base Price, Inflation, Maker)
    base_price = car_model_data[car_model_data["Model"] == model_name]["Base_Price_PHP"].values[0]
    inflation_rate = car_model_data[car_model_data["Model"] == model_name]["Monthly_Inflation_Rate"].values[0]
    car_maker = car_model_data[car_model_data["Model"] == model_name]["Make"].values[0]  # Get Car Maker

    # Get modification costs for selected parts
    modification_cost = car_modifications_data[(car_modifications_data["Model"] == model_name) &
                                               (car_modifications_data["Car_Part"].isin(selected_parts))]
    total_modification_cost = modification_cost["Modification_Cost_PHP"].sum()

    # Apply inflation over the specified months
    future_inflation = (1 + inflation_rate) ** months
    
    # Scale the input features
    scaled_features = scaler.transform([[base_price * future_inflation, total_modification_cost, inflation_rate]])
    
    # Predict price using trained model
    predicted_price = model.predict(scaled_features)[0]

    # Display full prediction results
    result = {
        "Car Maker": car_maker,  # ✅ Added Car Maker
        "Car Model": model_name,
        "Selected Car Parts": selected_parts,
        "Car Base Price (PHP)": base_price,
        "Car Parts Price (PHP)": total_modification_cost,
        "Current Total Price (PHP)": base_price + total_modification_cost,
        "Predicted Total Price After {} Months (PHP)".format(months): predicted_price
    }
    
    print("\n✅ Prediction Result:")
    for key, value in result.items():
        print(f"{key}: {value}")
    
    return result




In [ ]:
## Cell 7: Test the prediction function
# Test the prediction function
test_prediction = predict_price("Vios", ["Spoiler", "Body Kit"], months=12)
